# Interpolação de Dados com Kriging

## 1. Importações e Configuração
Nesta seção, importaremos as bibliotecas necessárias para o desenvolvimento do modelo de Kriging.

In [1]:
import pandas as pd
import numpy as np
from pykrige.ok import OrdinaryKriging
import folium
from folium.plugins import HeatMap
import logging

# Ativando o logging
logging.basicConfig(level=logging.INFO)

## 2. Carregamento dos Dados
Carregaremos os dados de pontos geoespaciais e de proxies do arquivo CSV.

In [2]:
def load_data(file_path):
    try:
        data = pd.read_csv(file_path)
        logging.info('Dados carregados com sucesso.')
        return data
    except Exception as e:
        logging.error(f'Falha ao carregar dados: {e}')
        return None

# Exemplo de uso
data = load_data('dataset/points.csv')
display(data.head())

INFO:root:Dados carregados com sucesso.


,age,basin,lat,long,climate,paleontology_weight,lithology_weight,palynomorphs_weight,geochemistry_weight
0,145,camamu-almada,-13.83,-38.5,h,3,2,1,4
1,145,camamu-almada,-13.83,-38.5,h,0,4,3,2
2,145,camamu-almada,-13.83,-38.5,h,2,1,0,4
3,140,araripe,-7.50,-39.5,h,4,3,1,0
4,140,Camamu-almada,-13.83,-38.5,h,1,0,2,3


## 3. Interpolação com Kriging
Usaremos a técnica de Kriging para interpolar os valores dos proxies com base nos pesos atribuídos.

In [3]:
def kriging_interpolation(x, y, weights, grid_res=100):
    try:
        # Remove any rows with missing values
        valid_mask = ~np.isnan(x) & ~np.isnan(y) & ~np.isnan(weights)
        x = x[valid_mask]
        y = y[valid_mask]
        weights = weights[valid_mask]

        if len(x) == 0 or len(y) == 0 or len(weights) == 0:
            raise ValueError("Input arrays must not be empty after removing missing values.")

        grid_x = np.linspace(x.min(), x.max(), grid_res)
        grid_y = np.linspace(y.min(), y.max(), grid_res)

        OK = OrdinaryKriging(x, y, weights, variogram_model='spherical')
        z, _ = OK.execute('grid', grid_x, grid_y)
        return grid_x, grid_y, z
    except Exception as e:
        logging.error(f'Erro durante a interpolação Kriging: {e}')
        return None, None, None

## 4. Visualização do Mapa de Calor
Criamos um mapa de calor interativo para visualizar a interpolação de Kriging.

In [4]:
def calculate_total_weight(data):
    return data[['paleontology_weight', 'lithology_weight', 'palynomorphs_weight', 'geochemistry_weight']].sum(axis=1)

def visualize_heatmap(grid_x, grid_y, z, data, total_weight):
    try:
        heat_data = [[grid_y[j], grid_x[i], z[j, i]] for i in range(len(grid_x)) for j in range(len(grid_y))]

        mapa = folium.Map(location=[0, 0], zoom_start=2)
        HeatMap(heat_data, radius=10, blur=15, max_zoom=1).add_to(mapa)

        for lat, lon, valor in zip(data['lat'], data['long'], total_weight):
            folium.CircleMarker(
                location=[lat, lon], radius=5, color='blue', fill=True,
                fill_color='blue', fill_opacity=0.6, popup=f'Valor Proxy: {valor}'
            ).add_to(mapa)

        return mapa
    except Exception as e:
        logging.error(f'Erro durante a visualização: {e}')
        return None#%% md
## 6. Mapa com Pontos e Interpolação
# Carregar os dados
data = load_data('dataset/points.csv')


# Verificar se os dados foram carregados corretamente
if data is not None:
    # Realizar a interpolação Kriging
    grid_x, grid_y, z = kriging_interpolation(data['long'], data['lat'], calculate_total_weight(data))

    # Verificar se a interpolação foi bem-sucedida
    if grid_x is not None and grid_y is not None and z is not None:
        # Visualizar o mapa de calor com os pontos originais
        mapa = visualize_heatmap(grid_x, grid_y, z, data, calculate_total_weight(data))
        display(mapa)
    else:
        logging.error('Falha na interpolação Kriging.')
else:
    logging.error('Falha ao carregar os dados.')

INFO:root:Dados carregados com sucesso.


## 5. Conclusão
O modelo de interpolação de Kriging foi implementado com sucesso, permitindo a visualização de dados geoespaciais.